# Tickers Module

This notebook demonstrates the ticker utilities from `specparser.amt.tickers`.

## What is a Ticker Schema?

A **ticker schema** is a template that defines all the data sources (tickers) needed for an asset:
- **Market tickers**: Price data (e.g., `AAPL US Equity` with field `PX_LAST`)
- **Vol tickers**: Implied volatility data (Near and Far month)
- **Hedge tickers**: Hedging instrument data

## Ticker Sources

| Source | Description |
|--------|-------------|
| `BBG` | Bloomberg direct ticker |
| `BBGfc` | Bloomberg futures spec (needs expansion) |
| `CV` | CitiVelocity (for swaption vols) |
| `calc` | Computed tickers (for swap hedges) |
| `nBBG` | Normalized Bloomberg (not in chain) |

## Split Ticker Format

When tickers change over time (e.g., LIBOR → SOFR transition):
```
ticker1:YYYY-MM:ticker2
```
- Before the date: use `ticker1`
- After the date: use `ticker2`

In [ ]:
# Setup: Imports and helper function
import pandas as pd
from specparser.amt import (
    # Ticker schemas
    get_tschemas,
    find_tschemas,
    # Futures expansion
    fut_spec2ticker,
    fut_norm2act,
    fut_act2norm,
    clear_normalized_cache,
    # Split tickers
    _split_ticker,
    # Period tickers
    find_tickers,
    find_tickers_ym,
    # Straddle tickers
    asset_straddle_tickers,
    # Table utilities
    table_to_rows,
    table_column,
    print_table,
)

# Path to AMT file
AMT_PATH = "../data/amt.yml"

def show(table):
    """Display a table as a pandas DataFrame for pretty rendering."""
    t = table_to_rows(table) if table.get("orientation") == "column" else table
    return pd.DataFrame(t["rows"], columns=t["columns"])

---
## 2. Ticker Schemas

### `get_tschemas(path, underlying)`

Get all tickers for a single asset. Returns a table with columns:
- `asset`: The underlying asset
- `cls`: Asset class (Commodity, Equity, Currency, etc.)
- `type`: Ticker type (Market, Vol, Hedge)
- `param`: Parameter name (e.g., Near, Far, hedge, hedge1)
- `source`: Data source (BBG, BBGfc, CV, calc)
- `ticker`: The ticker symbol or spec
- `field`: The data field to fetch

In [ ]:
# Get tickers for a single commodity asset
tickers = get_tschemas(AMT_PATH, "CL Comdty")
show(tickers)

In [ ]:
# Get tickers for an equity asset
tickers = get_tschemas(AMT_PATH, "SPX Index")
show(tickers)

In [ ]:
# Get tickers for a currency asset
tickers = get_tschemas(AMT_PATH, "EURUSD Curncy")
show(tickers)

### `find_tschemas(path, pattern, live_only)`

Find ticker schemas for multiple assets matching a regex pattern.

In [ ]:
# Find tickers for all commodities
commodity_tickers = find_tschemas(AMT_PATH, pattern="Comdty$", live_only=True)
print(f"Found {len(commodity_tickers['rows'])} ticker rows for commodities")
show(commodity_tickers)

In [ ]:
# Find tickers for specific assets
tickers = find_tschemas(AMT_PATH, pattern="^(CL|GC|SI) ", live_only=True)
print(f"CL, GC, SI tickers: {len(tickers['rows'])} rows")
show(tickers)

---
## 3. Understanding Ticker Types

### Market Tickers
- **Source**: Always `BBG`
- **Type**: `Market`
- **Purpose**: Get market price data
- **Field**: Usually `PX_LAST`

### Vol Tickers
- **Sources**: `BBG`, `CV` (CitiVelocity), `BBG_LMEVOL`
- **Type**: `Vol`
- **Params**: `Near` (near-month vol), `Far` (far-month vol)
- **Purpose**: Get implied volatility for option pricing

### Hedge Tickers
- **Sources**: `BBG` (nonfut), `BBGfc` (futures), `calc` (computed), `cds`
- **Type**: `Hedge`
- **Params**: `hedge`, `hedge1`, `hedge2`, `hedge3`, `fut`
- **Purpose**: Get prices for hedging instruments

In [ ]:
# Analyze ticker types for an asset
tickers = get_tschemas(AMT_PATH, "CL Comdty")

print("Ticker breakdown for CL Comdty:")
print()
for row in tickers["rows"]:
    asset, cls, typ, param, source, ticker, field = row
    print(f"Type: {typ:8s} | Param: {param:8s} | Source: {source:6s}")
    print(f"  Ticker: {ticker}")
    print(f"  Field:  {field}")
    print()

In [ ]:
# Count ticker types across all live assets
all_tickers = find_tschemas(AMT_PATH, pattern=".", live_only=True)

from collections import Counter
types = Counter(row[2] for row in all_tickers["rows"])  # type is column 2
sources = Counter(row[4] for row in all_tickers["rows"])  # source is column 4

print("Ticker types:")
for t, count in types.most_common():
    print(f"  {t}: {count}")

print("\nTicker sources:")
for s, count in sources.most_common():
    print(f"  {s}: {count}")

---
## 4. Futures Ticker Expansion

### `fut_spec2ticker(spec, year, month)`

Futures tickers use a spec string that needs to be expanded to actual contract tickers.

**Spec format:**
```
fut_code:CL,fut_month_map:GHJKMNQUVXZF,min_year_offset:0,market_code:Comdty
```

**Month map:** 12 characters mapping Jan-Dec to contract codes:
- Standard: `FGHJKMNQUVXZ` (F=Jan, G=Feb, H=Mar, J=Apr, K=May, M=Jun, N=Jul, Q=Aug, U=Sep, V=Oct, X=Nov, Z=Dec)

In [ ]:
# Example: Expand a crude oil futures spec
spec = "fut_code:CL,fut_month_map:GHJKMNQUVXZF,min_year_offset:0,market_code:Comdty"

# Get ticker for June 2024
ticker = fut_spec2ticker(spec, 2024, 6)
print(f"June 2024: {ticker}")

# Get ticker for December 2024
ticker = fut_spec2ticker(spec, 2024, 12)
print(f"December 2024: {ticker}")

In [ ]:
# Generate a full year of futures tickers
spec = "fut_code:CL,fut_month_map:GHJKMNQUVXZF,min_year_offset:0,market_code:Comdty"

print("CL Comdty futures calendar for 2024:")
print("-" * 40)
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
for month_num, month_name in enumerate(months, 1):
    ticker = fut_spec2ticker(spec, 2024, month_num)
    print(f"{month_name:3s}: {ticker}")

### Understanding Month Maps

Different assets have different delivery months. The month map encodes this:

| Position | Standard | Meaning |
|----------|----------|---------|
| 1 (Jan)  | F        | January contract |
| 2 (Feb)  | G        | February contract |
| 3 (Mar)  | H        | March contract |
| ...      | ...      | ... |
| 12 (Dec) | Z        | December contract |

Some assets only trade certain months (e.g., quarterly). In these cases, the month map repeats characters to indicate which contract to use.

In [ ]:
# Example: An asset with quarterly contracts only (H, M, U, Z)
# The month map might look like: HHHMMMUUUZZZ
# - Jan, Feb, Mar all use H (March)
# - Apr, May, Jun all use M (June)
# - Jul, Aug, Sep all use U (September)
# - Oct, Nov, Dec all use Z (December)

quarterly_spec = "fut_code:ED,fut_month_map:HHHMMMUUUZZZ,min_year_offset:0,market_code:Comdty"

print("Quarterly contract example:")
for month_num, month_name in enumerate(months, 1):
    ticker = fut_spec2ticker(quarterly_spec, 2024, month_num)
    print(f"{month_name:3s}: {ticker}")

---
## 5. Normalized vs Actual Tickers

Bloomberg uses specific ticker formats that may differ from the "normalized" format we generate.

**Example:**
- Normalized: `CLN2024 Comdty`
- Actual BBG: `CL N24 Comdty`

The `fut_norm2act()` and `fut_act2norm()` functions convert between these formats using a CSV lookup table.

**Note:** These functions require a chain CSV file with columns `normalized_future` and `actual_future`.

In [ ]:
# Check if chain CSV exists
import os

CHAIN_CSV = "../data/current_bbg_chain_data.csv"

if os.path.exists(CHAIN_CSV):
    print(f"Chain CSV found: {CHAIN_CSV}")
    
    # Example conversion
    normalized = "CLN2024 Comdty"
    actual = fut_norm2act(CHAIN_CSV, normalized)
    print(f"Normalized: {normalized}")
    print(f"Actual:     {actual}")
    
    # Reverse conversion
    if actual:
        back = fut_act2norm(CHAIN_CSV, actual)
        print(f"Back:       {back}")
else:
    print(f"Chain CSV not found: {CHAIN_CSV}")
    print("Skipping normalized/actual conversion examples.")
    print()
    print("To use these functions, you need a CSV with columns:")
    print("  normalized_future, actual_future")

In [ ]:
# Clear the lookup cache if needed
clear_normalized_cache()
print("Normalized/actual cache cleared")

---
## 6. Split Tickers (Ticker Transitions)

When Bloomberg changes ticker symbols, we use "split tickers" to handle the transition:

```
ticker1:YYYY-MM:ticker2
```

The `_split_ticker()` function expands this into two rows with date constraints:
- `param<YYYY-MM` - use ticker1 before the date
- `param>YYYY-MM` - use ticker2 after the date

In [ ]:
# Example: LIBOR to SOFR transition
split_ticker = "USSWAP5 CMPN Curncy:2023-06:USOSFR5 Curncy"
param = "hedge1"

result = _split_ticker(split_ticker, param)

print(f"Split ticker: {split_ticker}")
print(f"Original param: {param}")
print()
print("Expanded to:")
for ticker, new_param in result:
    print(f"  Ticker: {ticker}")
    print(f"  Param:  {new_param}")
    print()

In [ ]:
# A non-split ticker returns unchanged
regular_ticker = "SPY US Equity"
result = _split_ticker(regular_ticker, "hedge")

print(f"Regular ticker: {regular_ticker}")
print(f"Result: {result}")
print("(Single element list - no split)")

### Date Constraints in Params

After expansion, params have date constraints:
- `hedge<2023-06` - Valid **before** June 2023
- `hedge>2023-06` - Valid **after** June 2023
- `hedgeX2023-06` - Valid **exactly** in June 2023

These constraints are filtered when getting tickers for specific periods.

---
## 7. Tickers for Specific Periods

### `find_tickers_ym(path, pattern, live_only, year, month, chain_csv)`

Get tickers for a specific year/month. This:
1. Expands BBGfc specs to actual futures tickers
2. Applies date constraints from split tickers
3. Optionally converts normalized to actual tickers via chain CSV

In [ ]:
# Get tickers for CL Comdty in June 2024
tickers = find_tickers_ym(AMT_PATH, "^CL Comdty$", True, 2024, 6, None)
print(f"CL Comdty tickers for June 2024: {len(tickers['rows'])} rows")
show(tickers)

In [ ]:
# Compare with tschemas (unexpanded)
tschemas = get_tschemas(AMT_PATH, "CL Comdty")
print("Tschemas (unexpanded):")
show(tschemas)

### `find_tickers(path, pattern, live_only, start_year, end_year, chain_csv)`

Get unique tickers across a year range. Useful for knowing all tickers needed for a backtest period.

In [ ]:
# Get all unique tickers for commodities in 2024
tickers = find_tickers(
    AMT_PATH, 
    pattern="Comdty$", 
    live_only=True,
    start_year=2024, 
    end_year=2024,
    chain_csv=None
)
print(f"Unique commodity tickers for 2024: {len(tickers['rows'])} rows")
show(tickers)

---
## 8. Straddle Tickers

### `asset_straddle_tickers(asset, year, month, i, amt_path, chain_path)`

Get tickers for a specific straddle component with filtering:

**Filtering Rules:**
- **Market**: Always excluded (not needed for straddle valuation)
- **Vol/Near**: Kept only if straddle entry code (`ntrc`) is `"N"`, param → `"vol"`
- **Vol/Far**: Kept only if straddle entry code (`ntrc`) is `"F"`, param → `"vol"`
- **Hedge**: Always kept

**Parameters:**
- `i`: Straddle component index (0, 1, 2, ... for multiple schedule components)

In [ ]:
# Get straddle tickers for CL Comdty, June 2024, first component
try:
    tickers = asset_straddle_tickers(
        asset="CL Comdty",
        year=2024,
        month=6,
        i=0,  # First straddle component
        amt_path=AMT_PATH,
        chain_path=None
    )
    print("Straddle tickers for CL Comdty, June 2024, component 0:")
    show(tickers)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Compare different straddle components (i=0 vs i=1)
try:
    for i in range(2):
        tickers = asset_straddle_tickers(
            asset="CL Comdty",
            year=2024,
            month=6,
            i=i,
            amt_path=AMT_PATH,
            chain_path=None
        )
        print(f"Component {i}: {len(tickers['rows'])} ticker rows")
        # Show the straddle string
        if tickers['rows']:
            straddle = tickers['rows'][0][1]  # straddle is column 1
            print(f"  Straddle: {straddle}")
        print()
except Exception as e:
    print(f"Error: {e}")

---
## 9. Practical Examples

### Example 1: Analyze Ticker Sources by Asset Class

In [ ]:
# Count ticker sources by asset class pattern
patterns = [
    ("Comdty$", "Commodities"),
    ("Index$", "Indices"),
    ("Curncy$", "Currencies"),
    ("Equity$", "Equities"),
]

print("Ticker sources by asset class:")
print("-" * 50)

for pattern, name in patterns:
    tickers = find_tschemas(AMT_PATH, pattern, live_only=True)
    if tickers["rows"]:
        sources = Counter(row[4] for row in tickers["rows"])
        print(f"\n{name}:")
        for source, count in sources.most_common():
            print(f"  {source}: {count}")

### Example 2: Find Assets with Split Tickers

In [ ]:
# Find assets that have ticker transitions
all_tickers = find_tschemas(AMT_PATH, ".", live_only=True)

split_assets = set()
for row in all_tickers["rows"]:
    ticker = row[5]  # ticker column
    # Check for split ticker format: contains : and looks like a date
    if ":" in ticker and any(c.isdigit() for c in ticker):
        # More specific: check for YYYY-MM pattern
        import re
        if re.search(r':\d{4}-\d{2}:', ticker):
            split_assets.add(row[0])

if split_assets:
    print(f"Assets with split tickers ({len(split_assets)}):")
    for asset in sorted(split_assets):
        print(f"  {asset}")
else:
    print("No assets with split tickers found")

### Example 3: Build a Ticker Inventory

In [ ]:
# Build a summary of all unique tickers needed
all_tickers = find_tschemas(AMT_PATH, ".", live_only=True)

# Group by source
by_source = {}
for row in all_tickers["rows"]:
    source = row[4]
    ticker = row[5]
    if source not in by_source:
        by_source[source] = set()
    by_source[source].add(ticker)

print("Ticker inventory summary:")
print("-" * 40)
for source in sorted(by_source.keys()):
    tickers = by_source[source]
    print(f"\n{source}: {len(tickers)} unique tickers")
    # Show first 3 examples
    for t in list(tickers)[:3]:
        print(f"  - {t}")
    if len(tickers) > 3:
        print(f"  ... and {len(tickers) - 3} more")

---
## 10. Summary

### Ticker Schema Functions

| Function | Description |
|----------|-------------|
| `get_tschemas(path, underlying)` | Get ticker schemas for one asset |
| `find_tschemas(path, pattern, live_only)` | Find ticker schemas by regex pattern |

### Futures Expansion Functions

| Function | Description |
|----------|-------------|
| `fut_spec2ticker(spec, year, month)` | Convert BBGfc spec to ticker |
| `fut_norm2act(csv, ticker)` | Normalized → actual ticker |
| `fut_act2norm(csv, ticker)` | Actual → normalized ticker |
| `clear_normalized_cache()` | Clear lookup cache |

### Split Ticker Functions

| Function | Description |
|----------|-------------|
| `_split_ticker(ticker, param)` | Expand split ticker format |

### Period-Specific Functions

| Function | Description |
|----------|-------------|
| `find_tickers_ym(...)` | Get tickers for one month |
| `find_tickers(...)` | Get unique tickers for year range |
| `asset_straddle_tickers(...)` | Get filtered tickers for a straddle |

### Schema Columns

| Column | Description |
|--------|-------------|
| `asset` | Asset underlying |
| `cls` | Asset class |
| `type` | Market, Vol, or Hedge |
| `param` | Parameter name (Near, Far, hedge, etc.) |
| `source` | Data source (BBG, BBGfc, CV, calc) |
| `ticker` | Ticker symbol or spec |
| `field` | Data field to fetch |